**Clasificación** es una tarea que implica organizar objetos sistemáticamente en grupos o categorías apropiadas según las características que definen tales agrupaciones. Es importante enfatizar que los grupos están predefinidos de acuerdo con criterios establecidos. En nuestro caso, el uso de la **clasificación** es determinar la categoría a la que pertenece una observación no vista, dependiendo de la información de un conjunto de datos de **entrenamiento** con etiquetas apropiadas. Por lo tanto, la clasificación es una tarea de **aprendizaje supervisado**.

# KNN: K-nearest neighbors
Sirve esencialmente para clasificar valores, buscando los puntos de datos "más similares" (por cercanía).

La idea es realmente sencilla, el algoritmo clasifica cada dato nuevo en el grupo que corresponda, según tenga $k$ vecinos más cerca de un grupo o de otro. Es decir, calcula la distancia del elemento nuevo a cada uno de los patrones del conjunto de entrenamiento, y ordena dichas distancias de menor a mayor para ir seleccionando el grupo al que pertenece según votación.


## Preparación de los datos
En los siguientes bloques vamos a cargar y preparar los datos antes de implementar el algoritmo KNN.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Cargar el dataset de pingüinos
data = pd.read_csv("penguins.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


Como podemos apreciar, el *dataset* contiene algunos nulos. Procedamos a eliminarlos y aleatorizar los registros. Esto último nos permitirá estar seguros de que no siempre probaremos con los mismos patrones.

In [ ]:
# Eliminar filas con valores NaN
data = data.dropna().reset_index(drop=True)

# Aleatorizar el dataset
# frac: permite especificar la fracción del dataframe a retornar 0 -> 1
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data.isnull().sum().sum()

0

A continuación, codificaremos la variable objetivo (*target*) para proceder a seleccionar 20 registros por cada clase y como resultado crear nuestros conjuntos de datos de entrenamiento y prueba.

In [ ]:
# Separar 20 patrones por clase para entrenamiento
train_data = pd.DataFrame()
for species in data['species'].unique():
    train_data = pd.concat([train_data, data[data['species'] == species].head(20)])

test_data = data.drop(train_data.index) # Aqui está el resto de los patrones no seleccionados
train_data['species'].value_counts()

species
Adelie       20
Chinstrap    20
Gentoo       20
Name: count, dtype: int64

Finalmente seleccionamos solo las columnas `'bill_length_mm'`, `'bill_depth_mm'`, `'flipper_length_mm'` y `'body_mass_g'` para nuestros datasets con valores numéricos `X_train` y `X_test` y vectores con etiquetas de clase `y_train` y `y_test`.

In [ ]:
# Separar características (X) y etiquetas (y) de entrenamiento y prueba
X_train = train_data.select_dtypes(include='float64').values
y_train = train_data['species'].values

X_test = test_data.select_dtypes(include='float64').values
y_test = test_data['species'].values

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60, 4) (60,)
(273, 4) (273,)


## Implementación del KNN
Como mencionamos, para medir **similitud** requerimos de una métrica de distancia como la **Distancia Euclideana**.

La fórmula generalizada de la distancia euclidiana en un espacio de $n$ dimensiones entre dos puntos $A(x_1, x_2, \ldots, x_n)$ y $B(y_1, y_2, \ldots, y_n)$ es:

$d = \sqrt{(y_1 - x_1)^2 + (y_2 - x_2)^2 + \ldots + (y_n - x_n)^2} = \sqrt{\sum_{i=1}^{n} (y_i - x_i)^2}$

Donde:
* $ d $ es la distancia euclidiana entre los puntos $A$ y $B$.
* $ (x_1, x_2, \ldots, x_n)$ son las coordenadas del punto $A$.
* $ (y_1, y_2, \ldots, y_n)$ son las coordenadas del punto $B$.

Esta fórmula se aplica a cualquier número de dimensiones $n$.

In [ ]:
# Función para calcular la distancia euclidiana entre dos puntos
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

Para implementar el **KNN** utilizaremos la función `knn_predict`con los siguientes parámetros:
- **`X_train`**: Un conjunto de datos de entrenamiento, donde cada fila representa un ejemplo y cada columna representa una característica.
- **`y_train`**: Las etiquetas correspondientes a los ejemplos en `X_train`. Es un vector que indica la clase de cada ejemplo.
- **`X_test`**: Un conjunto de datos de prueba, donde se realizarán las predicciones. Al igual que `X_train`, cada fila representa un ejemplo y cada columna representa una característica.
- **`k`**: El número de vecinos más cercanos a considerar para hacer la predicción.

### Proceso:
1. **Inicialización de Predicciones**:
   ```python
   predictions = []
   ```
   Se crea una lista vacía `predictions` que almacenará las predicciones para cada ejemplo en `X_test`.

2. **Iteración sobre cada ejemplo en `X_test`**:
   ```python
   for x in X_test:
   ```
   Se itera sobre cada ejemplo `x` en el conjunto de prueba.

3. **Cálculo de distancias**:
   ```python
   distances = [euclidean_distance(x, x_train) for x_train in X_train]
   ```
   Para cada ejemplo `x`, se calcula la distancia euclidiana a todos los ejemplos en `X_train`. Se asume que existe una función `euclidean_distance` que calcula la distancia entre dos puntos.

4. **Obtención de los índices de los k vecinos más cercanos**:
   ```python
   nearest_indices = np.argsort(distances)[:k]
   ```
   Se ordenan las distancias y se obtienen los índices de los `k` ejemplos más cercanos en `X_train` utilizando `np.argsort`, que devuelve los índices que ordenarían la lista de distancias.

5. **Obtención de las etiquetas de los vecinos más cercanos**:
   ```python
   nearest_labels = [y_train[i] for i in nearest_indices]
   ```
   Se extraen las etiquetas correspondientes a los índices de los `k` vecinos más cercanos.

6. **Contar las etiquetas y obtener la más común**:
   ```python
   most_common = Counter(nearest_labels).most_common(1)[0][0]
   ```
   Se utiliza `Counter(nearest_labels).most_common(1)[0][0]` para encontrar la etiqueta que aparece con más frecuencia, es decir, la clase más común entre los vecinos.

7. **Almacenar la predicción**:
   ```python
   predictions.append(most_common)
   ```
   Se añade la etiqueta más común a la lista de `predictions`.

8. **Devolver las predicciones**:
   ```python
   return predictions
   ```
   Finalmente, la función devuelve la lista de predicciones para todos los ejemplos en `X_test`.

In [ ]:
# Función para predecir la etiqueta de una instancia de prueba
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for x in X_test:
        # Calcular distancias solo usando las columnas numéricas
        distances = [euclidean_distance(x, x_train) for x_train in X_train]
        # Obtener los índices de los k vecinos más cercanos
        nearest_indices = np.argsort(distances)[:k]
        # Obtener las etiquetas de los k vecinos más cercanos
        nearest_labels = [y_train[i] for i in nearest_indices]
        # Contar las etiquetas y obtener la más común
        most_common = Counter(nearest_labels).most_common(1)[0][0] # Get the mode
        predictions.append(most_common)
    return predictions

## Pruebas con diferentes $k$
Se pide probar con diferentes valores del hiperparámetro $k$ para determinar el mejor valor. Por lo tanto utilizaremos cantidades impares y utilizaremos una métrica básica de evaluación que nos indicará el porcentaje de aciertos.

In [ ]:
# Probar con diferentes valores de K
k_values = [1, 3, 5, 7, 9]

# Crear una lista de diccionarios para comparar y_test y predictions
comparison_data = []
comparison_df = pd.DataFrame(y_test, columns=['Species'])
for k in k_values:
    predictions = knn_predict(X_train, y_train, X_test, k)

    # Calcular la precisión
    accuracy = np.mean(predictions == y_test)
    print(f"Accuracy (k={k}): {accuracy * 100:.2f}%")

    # Agregar los resultados
    comparison_df[f'y_pred_{k}'] = predictions

# Muestra el DataFrame de comparación
comparison_df

Accuracy (k=1): 66.30%
Accuracy (k=3): 67.40%
Accuracy (k=5): 63.37%
Accuracy (k=7): 64.84%
Accuracy (k=9): 64.47%


,Species,y_pred_1,y_pred_3,y_pred_5,y_pred_7,y_pred_9
0,Adelie,Adelie,Adelie,Adelie,Adelie,Adelie
1,Adelie,Chinstrap,Chinstrap,Chinstrap,Gentoo,Adelie
2,Adelie,Chinstrap,Chinstrap,Chinstrap,Chinstrap,Chinstrap
3,Adelie,Adelie,Chinstrap,Chinstrap,Chinstrap,Chinstrap
4,Adelie,Adelie,Adelie,Adelie,Adelie,Adelie
...,...,...,...,...,...,...
268,Gentoo,Adelie,Gentoo,Gentoo,Gentoo,Gentoo
269,Adelie,Chinstrap,Chinstrap,Chinstrap,Chinstrap,Chinstrap
270,Adelie,Chinstrap,Adelie,Adelie,Adelie,Chinstrap
271,Chinstrap,Chinstrap,Chinstrap,Chinstrap,Chinstrap,Chinstrap
